# Détermination des instants de changement de vitesse des véhicules

Les bases de données sont extraites de la plateforme du département des transports des Etats-Unis (www.its.dot.gov/data).

L'objectif est de déterminer les temsp de réponse des véhicules par rapport à un changement de vitesse du leader.

In [39]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

In [40]:
#on travaille avec des fichiers différentiables par leur numéro
RunNo = [5, 6, 9, 10, 12, 13]

In [41]:
#chargement des .csv
for i in RunNo:
    globals()["data{}".format(i)] = pd.read_csv('data/raw/data{}.csv'.format(i), sep = ',', decimal = '.', low_memory = False)

In [42]:
#création d'une colonne pour avoir la distance au leader pour chacun des véhicules
#on récupérera les distances au leader de chaque véhicule aux instants de changement de réponse
for i in RunNo:
    globals()['data{}'.format(i)]['distToLeader_f1'] = globals()['data{}'.format(i)].follower1_radar1 
    globals()['data{}'.format(i)]['distToLeader_f2'] = globals()['data{}'.format(i)].follower1_radar1 + globals()['data{}'.format(i)].follower2_radar1     
    globals()['data{}'.format(i)]['distToLeader_f3'] = globals()['data{}'.format(i)].follower1_radar1 + globals()['data{}'.format(i)].follower2_radar1 + globals()['data{}'.format(i)].follower3_radar1 
    globals()['data{}'.format(i)]['distToLeader_f4'] = globals()['data{}'.format(i)].follower1_radar1 + globals()['data{}'.format(i)].follower2_radar1 + globals()['data{}'.format(i)].follower3_radar1 + globals()['data{}'.format(i)].follower4_radar1 

## Temps de réponse des véhicules face à un changement de vitesse

In [54]:
#suppression des valeurs manquantes
for i in RunNo:
    for j in range(5):
        if j ==0:
            globals()['data{}'.format(i)] = globals()['data{}'.format(i)][globals()['data{}'.format(i)]
                                                                          .leader_GPS_CARMA_speed.isna() == False]
        else:
            globals()['data{}'.format(i)] = globals()['data{}'.format(i)]
            [globals()['data{}'.format(i)]['follower{}_GPS_CARMA_speed'.format(j)].isna() == False]


In [55]:
#refonte de l'index
for i in RunNo:
    globals()["data{}".format(i)] = globals()["data{}".format(i)].reset_index(drop = True)

In [56]:
#tri des valeurs par 'Time'
for i in RunNo:
    globals()['data{}'.format(i)] = globals()['data{}'.format(i)].sort_values(by = 'Time')

In [57]:
#on reprend les index
for i in RunNo:
    globals()['data{}'.format(i)] = globals()['data{}'.format(i)].reset_index(drop = True)

On doit non seulement prendre en compte les changements de vitesse mais aussi ne prendre en comtpe que les changements réels et non le bruit qui peut apparaître sur les données.

In [58]:
#création du df pour l'analyse statistique dont le script figure dans un autre fichier
stat_kernel = pd.DataFrame()
stat_kernel['id'] = []
stat_kernel['veh_position'] = []
stat_kernel['reaction_time'] = []
stat_kernel['time'] = []
stat_kernel['veh_mode'] = []

Les données sont bruitées, on souhaite donc tout d'abord les rendre exploitables. On va donc effectuer des moyennes de vitesse sur des plages de temps choisies en fonction de la fréquence d'échantillonnage. On trace aussi les écart-types des valeurs de vitesse moyennées, ce qui pourra être utile pour travailler sur les données par la suite. Les écarts-types permettent d'avoir des informations sur les variations de vitesse.

In [59]:
#h représente la plage de données sur laquelle les valeurs de vitesse vont être moyennées
#on choisit h = 1s, avec une fréquence de 10 Hz, ça représente 10 valeurs par seconde
h = 10

### Traitement des leader

La première étape consiste à déterminer les instants de changement de réponse pour les leader qui sont à l'origine d'un changement de vitesse des suiveurs.

In [60]:
#on définit les variables moyenne et écart type
for i in RunNo:
    globals()['mu{}_leader'.format(i)] = []
    globals()['sigma{}_leader'.format(i)] = []
    for k in range(len(globals()["data{}".format(i)].Time) - h):
        #on calcule la moyenne des vitesses sur la plage [i - 10 ; i + 10]
        globals()['mu{}_leader'.format(i)].append(np.mean([globals()["data{}".format(i)].leader_GPS_CARMA_speed[j] 
                                                           for j in range(k, k + h)]))

In [62]:
#on définit sigma pour tout élément de mu comme l'écart type des valeurs de vitesse moyennées sur la tranche de largeur h
for i in RunNo:
    for k in range(len(globals()["mu{}_leader".format(i)]) - h):
        globals()['sigma{}_leader'.format(i)].append(np.std([globals()["mu{}_leader".format(i)][j] for j in range(k, k + h)]))
        

In [63]:
#parfois, des vitesses négatives sont observées pour certains véhicules
#on règle ce probème pour tous les véhicules du peloton d'un coup
for i in RunNo:
    for f in range(5):
        if f == 0:
            for j in range(len(globals()['data{}'.format(i)].Time)):
                if globals()['data{}'.format(i)].leader_GPS_CARMA_speed[j] < 0:
                    globals()['data{}'.format(i)].leader_GPS_CARMA_speed[j] = 0 
        else:
            for j in range(len(globals()['data{}'.format(i)].Time)):
                if globals()['data{}'.format(i)]['follower{}_GPS_CARMA_speed'.format(f)][j] < 0:
                    globals()['data{}'.format(i)]['follower{}_GPS_CARMA_speed'.format(f)][j] = 0 

/Users/ladino/opt/anaconda3/envs/cacc/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ladino/opt/anaconda3/envs/cacc/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [64]:
#on calcule l'écart-type global des vitesses de chaque leader
for i in RunNo:
    globals()['dsigma_tot{}_leader'.format(i)] = []
    globals()['dsigma_tot{}_leader'.format(i)] = np.std([globals()['sigma{}_leader'.format(i)][j]
                                                         - globals()['sigma{}_leader'.format(i)][j - 1] for j in range(1, len(globals()['data{}'.format(i)].Time[:-2*h]))])

Par la suite, un changement de vitesse est caractérisé par l'enchaînement de deux états :
- une période de stabilité doit précédé le changement de vitesse, il faut ensuite définir la durée de la stabilité ;
- une évolution caractérisée par une variation importante de la vitesse pendant une période fixée.

In [72]:
#EVOLUTION
#dt ne doit pas être trop grand mais assez grand pour avoir le temps d'être une vraie accélération ou décélération (1s)
dt = 20*1

#STABILITE
#DT doit être assez grande (20s)
DT = 20*20

Des facteurs arrivent ensuite pour caractériser les états définis précédemment ; ils sont fonction des données.

In [73]:
#a est utilisé comme facteur pour la période de stabilité (valeur assez grande pour omettre les oscillations d'adaptation)
a = 0.75
#b est utilisé pour les variations pour les périodes d'évolution de vitesse
#la valeur peut être plus faible que celle de a étant donné que les instants choisis
#doivent donner des modifications de vitesse assez importantes pendant dt
b = 0.8
#c est utilisé pour qualifier la proportion de valeurs dans l'intervalle [j, j + dt] requise
d = 0.8

In [88]:
#remplissage des listes 'changement de vitesse'
#la liste contient les instants de cahngement de vitesse du leader pour l'enregsitrement 'i'
for i in RunNo:
    print(i)
    globals()['chgt_vitesse{}_leader'.format(i)] = []
    #on souhaite aussi avoir accès à la vitesse du leader à cet instant de changement de vitesse
    #des analyses peuvent être réalisées par rapport à la vitesse du leader par la suite
    globals()['vitesse{}_leader'.format(i)] = []
    inc_prov = -1
    #on commence à DT et on finit à dt
    for j in range(DT + 1, len(globals()['data{}'.format(i)].Time[:-2*h]) - dt):
        #on veut savoir si une évolution est possible
        #on doit tout d'abord observer une période stabilité pendant DT
        #on regarde la stabilité sur sigma et non sur mu
        if (max([abs(globals()['sigma{}_leader'.format(i)][k] - globals()['sigma{}_leader'.format(i)][k - 1])
                 for k in range(j - DT, j)]) < a*globals()['dsigma_tot{}_leader'.format(i)]):
            inc = 0
            #print(j, globals()['data{}'.format(i)]['elapsed_time (s)'][j])
            #on veut prendre en compte non pas le minimum qui est trop restrictif
            #on cherche à avoir 90% des valeurs de l'intervalle de largeur dt qui conviennent
            for k in range(j, j + dt):
                if (abs(globals()['sigma{}_leader'.format(i)][k + 1] - globals()
                        ['sigma{}_leader'.format(i)][k])) > (b*globals()['dsigma_tot{}_leader'.format(i)]):
                    inc += 1
            if inc > d*dt:
                if inc_prov == -1:
                    globals()['chgt_vitesse{}_leader'.format(i)].append(globals()['data{}'.format(i)].Time[j])
                    globals()['vitesse{}_leader'.format(i)].append(globals()['data{}'.format(i)].leader_GPS_CARMA_speed[j])
                    inc_prov += 1
                else:
                    #on supprime les changements de vitesse en fin d'enregistrements afin d'éviter de ne pas les retrouver
                    #pour les derniers suiveurs
                    if (globals()['data{}'.format(i)].Time[j] > globals()['chgt_vitesse{}_leader'.format(i)][inc_prov] + dt) and (globals()['data{}'.format(i)].Time[j] < (globals()['data{}'.format(i)].Time.iloc[-1] - 15)):
                        globals()['chgt_vitesse{}_leader'.format(i)].append(globals()['data{}'.format(i)].Time[j])
                        globals()['vitesse{}_leader'.format(i)].append(globals()['data{}'.format(i)].leader_GPS_CARMA_speed[j])
                        inc_prov += 1

5
6
9
10
12
13


### Traitement pour les suiveurs

On effectue le même travail pour les suiveurs, à la différence prêt que pour détecter un changement de vitesse pour un suiveur, il faut que celui-ci existe pour le leader.

In [84]:
#on définit les variables moyenne et écart type
for i in RunNo:
    for j in range(1, 5):
        globals()['mu{}_follower{}'.format(i, j)] = []
        globals()['sigma{}_follower{}'.format(i, j)] = []
        for k in range(len(globals()["data{}".format(i)].Time) - h):
            #on calcule la moyenne des vitesses sur la plage [i - 10 ; i + 10]
            globals()['mu{}_follower{}'.format(i, j)].append(np.mean([globals()["data{}".format(i)]
                                                                      ['follower{}_GPS_CARMA_speed'.format(j)][l] 
                                                                      for l in range(k, k + h)]))

In [19]:
#on définit sigma pour tout élément de mu comme l'écart type des valeurs de vitesse moyennées sur la tranche de largeur h
for i in RunNo:
    for j in range(1, 5):
        for k in range(len(globals()["mu{}_follower{}".format(i, j)]) - h):
            globals()['sigma{}_follower{}'.format(i, j)].append(np.std([globals()["mu{}_follower{}".format(i, j)][l] 
                                                                        for l in range(k, k + h)]))
        

In [20]:
for i in RunNo:
    for j in range(1, 5):
        globals()['dsigma_tot{}_follower{}'.format(i, j)] = []
        globals()['dsigma_tot{}_follower{}'.format(i, j)] = np.std([globals()['sigma{}_follower{}'.format(i, j)][k] - globals()['sigma{}_follower{}'.format(i, j)][k - 1] for k in range(1, len(globals()['data{}'.format(i)].Time[:-2*h]))])

In [21]:
#remplissage des listes 'changement de vitesse'
i_loc = 0
for i in RunNo:
    print(i)
    for f in range(1, 5):
        globals()['chgt_vitesse{}_follower{}'.format(i, f)] = []
        #on définit aussi la liste des écarts au leader et des écarts au prédécesseur pour des éventuelles analyses
        globals()['ecart_position_leader{}_follower{}'.format(i, f)] = []
        globals()['ecart_position_predec{}_follower{}'.format(i, f)] = []
        inc_prov = -1
        leader = 0
        #on commence à DT et on finit à dt
        for j in range(DT + 1, len(globals()['data{}'.format(i)].Time[:-2*h]) - dt):
            #on ne prendra en compte qu'un nombre fini de changements de vitesse en fonction de la taille de la liste
            #des changements de vitesse du leader
            if leader < len(globals()['chgt_vitesse{}_leader'.format(i)]):
                #on compare tout de suite à l'élément de la liste du leader afin de ne pas faire des itérations inutiles
                #et gagner en temps de calcul
                #on vérifie aussi que l'instant qu'on s'apprête à prendre ne se trouve pas à une distance trop éloignée
                #du leader, sinon on passe au suivant
                if globals()['data{}'.format(i)].Time[j] > globals()['chgt_vitesse{}_leader'.format(i)][leader] and (globals()['data{}'.format(i)].
                                                                                                                     Time[j] < globals()['chgt_vitesse{}_leader'.format(i)][leader] + 20):
                    #on n'a pas besoin d'observer la période de stabilité car on raisonne par rapport au leader
                    #on cherche le premier instant à partir duquel un changement notable de vitesse se produit
                    #on peut donc utiliser la comparaison du différentiel de sigma à un indicateur
                    if globals()['sigma{}_follower{}'.format(i, f)][j + 1] - globals()['sigma{}_follower{}'.
                                                                                       format(i, f)][j] > 0.5*b*globals()['dsigma_tot{}_follower{}'.format(i, f)]:
                        #on peut garder quand même la condition sur la durée du changement de vitesse afin de ne pas prendre
                        #un bruit qui ne serait pas un changement de vitesse
                        inc = 0
                        for k in range(j, j + dt):
                            if abs(globals()['sigma{}_follower{}'.format(i, f)][k + 1] - globals()['sigma{}_follower{}'.
                                                                                                   format(i, f)][k]) > 0.5*b*globals()['dsigma_tot{}_follower{}'.format(i, f)]:
                                inc += 1
                        if inc >= 0.75*d*dt:
                            if inc_prov == -1:
                                globals()['chgt_vitesse{}_follower{}'.format(i, f)].append(globals()['data{}'.format(i)].Time[j])
                                globals()['ecart_position_leader{}_follower{}'.format(i, f)].append(globals()['data{}'.format(i)]['distToLeader_f{}'.format(f)][j])
                                globals()['ecart_position_predec{}_follower{}'.format(i, f)].append(globals()['data{}'.format(i)]['follower{}_radar1'.format(f)][j])
                                inc_prov += 1
                                #stat_kernel
                                stat_kernel.loc[i_loc] = ['{}_{}'.format(i, f), f, globals()['data{}'.format(i)].
                                                          Time[j] - globals()['chgt_vitesse{}_leader'.format(i)][leader], globals()['data{}'.format(i)].Time[j], 1]
                                leader += 1
                                i_loc += 1
                            else:
                                if globals()['data{}'.format(i)].Time[j] > globals()['chgt_vitesse{}_follower{}'.format(i, f)][inc_prov] + dt:
                                    globals()['chgt_vitesse{}_follower{}'.format(i, f)].append(globals()['data{}'.format(i)].Time[j])
                                    globals()['ecart_position_leader{}_follower{}'.format(i, f)].append(globals()['data{}'.format(i)]['distToLeader_f{}'.format(f)][j])
                                    globals()['ecart_position_predec{}_follower{}'.format(i, f)].append(globals()['data{}'.format(i)]['follower{}_radar1'.format(f)][j])
                                    inc_prov += 1
                                    #stat_kernel
                                    stat_kernel.loc[i_loc] = ['{}_{}'.format(i, f), f, globals()['data{}'.format(i)].
                                                              Time[j] - globals()['chgt_vitesse{}_leader'.format(i)][leader], globals()['data{}'.format(i)].Time[j], 1]
                                    leader += 1
                                    i_loc += 1
                elif globals()['data{}'.format(i)].Time[j] > globals()['chgt_vitesse{}_leader'.format(i)][leader] + 20:
                    globals()['chgt_vitesse{}_follower{}'.format(i, f)].append(-1)
                    globals()['ecart_position_leader{}_follower{}'.format(i, f)].append(-1)
                    globals()['ecart_position_predec{}_follower{}'.format(i, f)].append(-1)
                    leader += 1

5
6
9
10
12
13


# Enregistrement en .csv

In [44]:
stat_kernel.to_csv('stat_kernel_CARMA.csv', index = False)